In [1]:
import pandas as pd
import numpy as np
import os

# Importing data

In [3]:
path = r"C:\Users\toloz\Instacart Basket Analysis 9_11_2024"

In [5]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared data', 'orders_products_merged_derived.pkl'))

## Step 2

In [7]:
ords_prods_merge.head(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,merge_indicator,price_range_loc,busiest day,busiest days,busiest_period_of_day
0,2539329,1,1,2,8,7.0,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,two slowest days,Average orders
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,two slowest days,Most orders
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Least busy,two slowest days,Average orders
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Least busy,two slowest days,Most orders


In [13]:
df = ords_prods_merge[:1000000]

In [15]:
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [17]:
ords_prods_merge.groupby(['department_id']).agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# Step 3

there is 21 department_id while in the subset there was 8 department_id and the mean for example for department_id 4 changed as others have changed because we looking into more data

# Step 4

In [30]:
# Creating the column "Max_order"
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [32]:
# Assign loyalty flag
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [26]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [28]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [34]:
# check loyalty flags 
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [38]:
ords_prods_merge[['user_id', 'loyalty_flag', 'order_number','max_order']].head(25)

,user_id,loyalty_flag,order_number,max_order
0,1,New customer,1,10
1,1,New customer,2,10
2,1,New customer,3,10
3,1,New customer,4,10
4,1,New customer,5,10
5,1,New customer,6,10
6,1,New customer,7,10
7,1,New customer,8,10
8,1,New customer,9,10
9,1,New customer,10,10


# Step 5

In [49]:
# Basic stats like mean, min, max
ords_prods_merge.groupby(['loyalty_flag']).agg({'prices':['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

New customers have the least price of products in both the average and max. Loyal customer have the highest price of products in both the average and max.

# Step 6

In [53]:
# Creating column for average spender
ords_prods_merge['average_spend']=ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [55]:
# flag for low and high spender
ords_prods_merge.loc[ords_prods_merge['average_spend']<10,'spender_flag']='Low_spender'

In [57]:
ords_prods_merge.loc[ords_prods_merge['average_spend']>= 10, 'spender_flag'] = 'High_spender'

In [59]:
# Check  flags
ords_prods_merge['spender_flag'].value_counts(dropna = False)

spender_flag
Low_spender     31770614
High_spender      634245
Name: count, dtype: int64

In [61]:
ords_prods_merge[['user_id', 'average_spend', 'spender_flag', 'prices']].head(30)

,user_id,average_spend,spender_flag,prices
0,1,6.367797,Low_spender,9.0
1,1,6.367797,Low_spender,9.0
2,1,6.367797,Low_spender,9.0
3,1,6.367797,Low_spender,9.0
4,1,6.367797,Low_spender,9.0
5,1,6.367797,Low_spender,9.0
6,1,6.367797,Low_spender,9.0
7,1,6.367797,Low_spender,9.0
8,1,6.367797,Low_spender,9.0
9,1,6.367797,Low_spender,9.0


# Step 7

In [64]:
# column for frequet users
ords_prods_merge['Customer_frequency'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [66]:
# flag on order frequency

In [68]:
ords_prods_merge.loc[ords_prods_merge['Customer_frequency'] > 20, 'frequency_flag'] = 'Non_frequent_customer'

In [70]:
ords_prods_merge.loc[(ords_prods_merge['Customer_frequency']>10) & (ords_prods_merge['Customer_frequency']<=20),'frequency_flag'] = 'Regular customer'

In [72]:
ords_prods_merge.loc[ords_prods_merge['Customer_frequency'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [74]:
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent Customer        22790523
Regular customer          6927608
Non_frequent_customer     2686728
Name: count, dtype: int64

In [76]:
#Nan is customers that only bought once 

In [78]:
ords_prods_merge[['user_id','days_since_prior_order', 'Customer_frequency', 'frequency_flag']].head(20)

,user_id,days_since_prior_order,Customer_frequency,frequency_flag
0,1,7.0,20.0,Regular customer
1,1,15.0,20.0,Regular customer
2,1,21.0,20.0,Regular customer
3,1,29.0,20.0,Regular customer
4,1,28.0,20.0,Regular customer
5,1,19.0,20.0,Regular customer
6,1,20.0,20.0,Regular customer
7,1,14.0,20.0,Regular customer
8,1,0.0,20.0,Regular customer
9,1,30.0,20.0,Regular customer


### EXPORT DATAFRAME

In [81]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'aggr_ords_prods_merge_deri.pkl'))